In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 209 plant S data

### Reading the files and extracting relevant information:

In [5]:
plant = "S"
xls_files = {}

xls = pd.ExcelFile(
    f"../../../../data/raw/209/EMBRAPII hubIC IACC_CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  S ['INSTRUÇÕES', 'Clínquer', '97', '62']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])

        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('S', '97', array(['CPIIF40'], dtype=object)),
 ('S', '62', array(['CPIIZ32'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPIIF40': 1, 'CPIIZ32': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Composição"].rename(
            {"Sulfato de cálcio": "Sulfato de cálcio (composição)"}, axis=1
        ),
        df["Cimento"]["Análise mineralógica"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [13]:
# df.info()

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [14]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
    # ("Unnamed: 139_level_1", "#400"),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
    "Clínquer": "Clinker",
    "Fíler calcário": "Calcareous filler",
    "Cinza volante": "Fly ash",
    "Escória": "Slag",
    "Argila calcinada": "Calcined clay",
    "Sulfato de cálcio": "Calcium sulfate",
    "C3S total": "Total C3S",
    "C2S total": "Total C2S",
    "C2S alpha": "Alpha C2S",
    "C2S beta": "Beta C2S",
    "C2S gama": "Gamma C2S",
    "C4AF": "C4AF",
    "C3A": "C3A",
    "C3A cubic": "Cubic C3A",
    "C3A orto": "Orthorhombic C3A",
    "CaO livre": "Free CaO",
    "Portlandita": "Portlandite",
    "Periclasio": "Periclase",
    "Arcanita": "Arcanite",
    "Aphthalita": "Aphthitalite",
    "Langbeinita": "Langbeinite",
    "Bassanita": "Bassanite",
    "Anidrita": "Anhydrite",
    "Calcita": "Calcite",
    "Dolomita": "Dolomite",
    "Quartzo": "Quartz",
    "Muscovita": "Muscovite",
    "Sulfato de cálcio (composição)": "Calcium sulfate (composition)",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [15]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

# Preprocessing
df["Cement_Type"] = (
    df["Cement_Type"]
    .str.replace("Expedido", "")
    .str.replace("Linha 9", "")
    .str.replace("Itajaí", "")
    .str.replace("D-G", "")
    .str.replace("D-E", "")
    .str.replace("DF2", "")
    .str.replace("CPIIF40", "CP II-F-40")
    .str.replace("CPIIF40", "CP II-F40")
    .str.replace("CP VARI", "CP V-ARI")
    .str.replace("CPVARI", "CP V-ARI")
    .str.replace("CP V-RSARI", "CP V-ARI RS")
    .str.replace("CP III-RS40", "CP III-40 RS")
    .str.replace("CP III-RS32", "CP III-32 RS")
    .str.replace("CP IV-RS32", "CP IV-32 RS")
    .str.replace("CP III40", "CP III-40")
    .str.replace("CP III32", "CP III-32")
    .str.replace("CP I-S40", "CP I-S-40")
    .str.replace("CP I40", "CP I-40")
    .str.replace("CP II-E32", "CP II-E-32")
    .str.replace("CP II-E40", "CP II-E-40")
    .str.replace("CP II-F FIBRO40", "CP II-F-40")
    .str.replace("CP II-F32", "CP II-F-32")
    .str.replace("CP II-F40", "CP II-F-40")
    .str.replace("CP II-Z32", "CP II-Z-32")
    .str.replace("CP II-Z40", "CP II-Z-40")
    .str.replace("CP IV32", "CP IV-32")
    .str.replace("CP IND", "CP I")
    .str.replace("CPINDCC", "CP I")
    .str.strip()
)

df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "Calcium sulfate (composition)",
    "Clinker",
    "Calcareous filler",
    "Fly ash",
    "Slag",
    "Calcined clay",
    "Calcium sulfate",
    "Total C3S",
    "Total C2S",
    "Alpha C2S",
    "Beta C2S",
    "Gamma C2S",
    "C4AF",
    "C3A",
    "Cubic C3A",
    "Orthorhombic C3A",
    "Free CaO",
    "Portlandite",
    "Periclase",
    "Arcanite",
    "Aphthitalite",
    "Langbeinite",
    "Bassanite",
    "Anhydrite",
    "Calcite",
    "Dolimita",
    "Quartz",
    "Muscovite",
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string 

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

In [18]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Calcite,100.000000
Cubic C3A,100.000000
Specific Gravity,100.000000
TiO2,100.000000
Muscovite,100.000000
Quartz,100.000000
Dolimita,100.000000
Anhydrite,100.000000
Calcium sulfate,100.000000
Langbeinite,100.000000


<IPython.core.display.Javascript object>

In [19]:
for var, val in (
    (df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).items()
):
    s = f"""<tr>
        <td>{var}</td>
        <td>{round(val, 2)}</td>
    </tr>"""
    if val >= 40:
        # print(s)
        print(f'"{var}",')

"Calcite",
"Cubic C3A",
"Specific Gravity",
"TiO2",
"Muscovite",
"Quartz",
"Dolimita",
"Anhydrite",
"Calcium sulfate",
"Langbeinite",
"Aphthitalite",
"Arcanite",
"Periclase",
"Orthorhombic C3A",
"Bassanite",
"CS1",
"Calcareous filler",
"Gamma C2S",
"Beta C2S",
"Alpha C2S",
"Total C2S",
"Total C3S",
"Calcium sulfate (composition)",
"Clinker",
"Calcined clay",
"C4AF",
"Slag",
"Fly ash",
"#325",
"Portlandite",
"C3A",


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant S - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Calcite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Cubic C3A</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Specific Gravity</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>TiO2</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Muscovite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Quartz</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Dolimita</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Anhydrite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Calcium sulfate</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Langbeinite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Aphthitalite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Arcanite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Periclase</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Orthorhombic C3A</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Bassanite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>CS1</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Calcareous filler</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Gamma C2S</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Beta C2S</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Alpha C2S</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Total C2S</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Total C3S</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Calcium sulfate (composition)</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Clinker</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Calcined clay</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>C4AF</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Slag</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Fly ash</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>#325</td>
        <td>93.72</td>
    </tr>
    <tr>
        <td>Portlandite</td>
        <td>77.12</td>
    </tr>
    <tr>
        <td>C3A</td>
        <td>76.32</td>
    </tr>
</table>

In [20]:
FEATRUES_TO_DROP = [
    "Calcite",
    "Cubic C3A",
    "Specific Gravity",
    "TiO2",
    "Muscovite",
    "Quartz",
    "Dolimita",
    "Anhydrite",
    "Calcium sulfate",
    "Langbeinite",
    "Aphthitalite",
    "Arcanite",
    "Periclase",
    "Orthorhombic C3A",
    "Bassanite",
    "CS1",
    "Calcareous filler",
    "Gamma C2S",
    "Beta C2S",
    "Alpha C2S",
    "Total C2S",
    "Total C3S",
    "Calcium sulfate (composition)",
    "Clinker",
    "Calcined clay",
    "C4AF",
    "Slag",
    "Fly ash",
    "#325",
    "Portlandite",
    "C3A",
]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Removing features with more than 23% of zeros

In [21]:
zero_values = {}
for col in df.select_dtypes(include="number").columns:
    zero_percentages = (df[df[col].eq(0)].shape[0] / df.shape[0]) * 100
    zero_values[col] = zero_percentages

zero_percentages = pd.Series(zero_values, name=f"Zero (%) for {plant}")
zero_percentages = zero_percentages.sort_values(ascending=False)
zero_percentages = zero_percentages.to_frame(name=f"Zero (%) for {plant}")
zero_percentages.style.background_gradient(cmap="Reds")

,Zero (%) for S
#200,90.740741
Free CaO,0.000000
Insoluble Residue,0.000000
CS7,0.000000
CS3,0.000000
Final setting time,0.000000
Initial setting time,0.000000
#400,0.000000
Blaine,0.000000
Loss on Ignition,0.000000


<IPython.core.display.Javascript object>

<h3>Percentage of zero values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>#200</td>
        <td>90.74</td>
    </tr>
</table>

In [22]:
FEATRUES_TO_DROP = ["#200"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [23]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [24]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [25]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

In [26]:
df.shape

(1495, 20)

<IPython.core.display.Javascript object>

In [27]:
# df.describe().T

<IPython.core.display.Javascript object>

In [28]:
# df_copy = df.copy()

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [29]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

In [30]:
df.shape

(1495, 20)

<IPython.core.display.Javascript object>

In [31]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Final setting time,1.337793
Initial setting time,1.270903
#400,0.869565
Blaine,0.802676
Al2O3,0.602007
Fe2O3,0.401338
CaO,0.401338
MgO,0.401338
Na2O,0.401338
SiO2,0.401338


<IPython.core.display.Javascript object>

## Dropping rows with increased missing values after preprocessing

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td></td>
        <td></td>
    </tr>
</table>

In [32]:
# FEATRUES_TO_DROP = ["C3A", "Aphthitalite"]
# df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

In [33]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Final setting time,1.337793
Initial setting time,1.270903
#400,0.869565
Blaine,0.802676
Al2O3,0.602007
Fe2O3,0.401338
CaO,0.401338
MgO,0.401338
Na2O,0.401338
SiO2,0.401338


<IPython.core.display.Javascript object>

In [34]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,1495,2022-03-28 07:47:09.431438080,2021-01-02 00:00:00,2021-08-13 00:00:00,2022-03-26 00:00:00,2022-11-04 00:00:00,2023-07-19 00:00:00,NaN
Free CaO,1491.0,1.481321,0.7,1.13,1.42,1.77,2.99,0.440323
CaO,1489.0,56.535401,43.527,53.054001,58.021999,60.000999,63.293999,3.74702
MgO,1489.0,4.96656,3.134,4.647,4.96994,5.317,7.173,0.489584
Na2O,1489.0,0.20271,0.125,0.179,0.197,0.224,0.491,0.033811
Al2O3,1486.0,4.829193,3.73,4.223,4.7735,5.398,6.944,0.623002
SiO2,1489.0,20.692523,16.054001,18.652,19.694,22.771999,25.888,2.124614
SO3,1489.0,2.744147,1.57,2.669,2.743,2.827,3.265,0.142467
K2O,1489.0,0.71673,0.351,0.566,0.714,0.868,1.076,0.172401
Fe2O3,1489.0,3.022008,2.267,2.894,3.032,3.149,3.879,0.176738


<IPython.core.display.Javascript object>

In [35]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,1495,NaN,NaN,NaN,2022-03-28 07:47:09.431438080,2021-01-02 00:00:00,2021-08-13 00:00:00,2022-03-26 00:00:00,2022-11-04 00:00:00,2023-07-19 00:00:00
Cement_Type,1495,2,CP II-F-40,749,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1495 entries, 0 to 1511
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  1495 non-null   datetime64[ns]
 1   Free CaO              1491 non-null   float64       
 2   CaO                   1489 non-null   float64       
 3   MgO                   1489 non-null   float64       
 4   Na2O                  1489 non-null   float64       
 5   Al2O3                 1486 non-null   float64       
 6   SiO2                  1489 non-null   float64       
 7   SO3                   1489 non-null   float64       
 8   K2O                   1489 non-null   float64       
 9   Fe2O3                 1489 non-null   float64       
 10  Loss on Ignition      1492 non-null   float64       
 11  Insoluble Residue     1490 non-null   float64       
 12  Blaine                1483 non-null   float64       
 13  #400                  1

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [37]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [38]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [39]:
df.to_csv("../../../../data/interim/209/s.csv", index=False)

<IPython.core.display.Javascript object>